# Principal file.

## Part 1: prepare for loop over all .txt files
1. import dependencies
2. set up Stanford dependency parser.
3. create the positive & negative word lists:  general_inquirer_import.ipynb
4. create functions for: 1: tokenize // 2: tokenize & stem
5. create Empty Lists for number of positive words, negative words, active voice, passive voice, etc.
## Part 2: LOOP - once for every .txt file
1. Sentiment analysis (open & tokenize text file) <br>
ii  Count aggregate number of all positive and negative items <br>
iii Show all pos and neg words for this text file. <br>
iv  construct set // frequency of all pos and neg words in all of the text files. <br>
2. Turn-taking <br>
ii.  No. participants <br>
iii. No. turns per speaker (i.e. just count lines starting with N1 etc) <br>
iv.  Per speaker: run another loop (sentiment3_each_speaker.ipynb) <br>

## Part 3: After loop - summarise results


### User Input

In [1]:
#### Words to match on
list_words = ["i", "you", "we", "decide", "decision", "option", "options"] 

#### Phrases to match on
list_phrases = []

#### Speaker identifiers (these are followed by a colon at the start of each turn)
speakers = ["P1", "N1", "D1", "D2", "R1", "R2"]

### import dependencies

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.parse.stanford import StanfordDependencyParser
from nltk.parse.stanford import StanfordParser
from bs4 import BeautifulSoup
import re
import os
import glob
import codecs
from sklearn import feature_extraction
import mpld3
from matplotlib import pyplot as plt
from itertools import zip_longest
import spacy
from collections import Counter
from nltk import sent_tokenize
from nltk import word_tokenize
import io
import pandas as pd
import xlrd
import pprint


#### configure to display results of all content of cells

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### display the environment variables

In [4]:
print(os.environ['CLASSPATH'])

C:\Users\Padhraig\OneDrive\Python\inca python code\Sentiment_Analysis;C:\Users\Padhraig\Documents\INCA data\inca python code\Sentiment_Analysis;C:\ProgramData\Anaconda3;C:\ProgramData\Anaconda3\Scripts;C:\ProgramData\Anaconda3\pkgs\python-3.6.2-h6679aeb_11;C:\ProgramData\Anaconda3\Lib\site-packages\stanford-corenlp-full-2017-06-09;C:\Users\Padhraig\OneDrive\Python\inca python code;C:\Users\Padhraig\OneDrive\Python\python-conversational-alignment-master\stanford-postagger-2015-04-20;


### Create containers for results. i.e. link participant identifiers with metrics.


E.g. if utterances / paragraphs start with: Doctor1
                                            Patient1
Create list as:
speaker_list = [Doctor1:, Patient1:]

In [5]:
metrics = ["Num_turns_", "Num_words_"]
### "speakers" list was produced already.

metrics_speaker = []

for a in metrics:
    for x in speakers:
        full = a + x
        metrics_speaker.append(full)

# Create dictionary
values = np.zeros(len(metrics_speaker))
metrics_speaker_dict  = dict(zip(metrics_speaker, values))

print("Print Dictionary Contents")
metrics_speaker_dict




Print Dictionary Contents


{'Num_turns_D1': 0.0,
 'Num_turns_D2': 0.0,
 'Num_turns_N1': 0.0,
 'Num_turns_P1': 0.0,
 'Num_turns_R1': 0.0,
 'Num_turns_R2': 0.0,
 'Num_words_D1': 0.0,
 'Num_words_D2': 0.0,
 'Num_words_N1': 0.0,
 'Num_words_P1': 0.0,
 'Num_words_R1': 0.0,
 'Num_words_R2': 0.0}

### spacy preparation

In [6]:
nlp = spacy.load('en')

In [7]:
doc = nlp("I cured the illness. The illness was cured by me. A red cup. I ran quickly.")

counter = 0

while counter < len(list(doc.sents)):
    for word in list(doc.sents)[counter]:
        print("Word:", word.text)     
        print("Tag:", word.tag_)
        print("Head:", word.head.text)
        print( "Dependency relation:", word.dep_)
        print( "Children:", list(word.children))
        print("")
    counter += 1


Word: I
Tag: PRP
Head: cured
Dependency relation: nsubj
Children: []

Word: cured
Tag: VBD
Head: cured
Dependency relation: ROOT
Children: [I, illness, .]

Word: the
Tag: DT
Head: illness
Dependency relation: det
Children: []

Word: illness
Tag: NN
Head: cured
Dependency relation: dobj
Children: [the]

Word: .
Tag: .
Head: cured
Dependency relation: punct
Children: []

Word: The
Tag: DT
Head: illness
Dependency relation: det
Children: []

Word: illness
Tag: NN
Head: cured
Dependency relation: nsubjpass
Children: [The]

Word: was
Tag: VBD
Head: cured
Dependency relation: auxpass
Children: []

Word: cured
Tag: VBN
Head: cured
Dependency relation: ROOT
Children: [illness, was, by, .]

Word: by
Tag: IN
Head: cured
Dependency relation: agent
Children: [me]

Word: me
Tag: PRP
Head: by
Dependency relation: pobj
Children: []

Word: .
Tag: .
Head: cured
Dependency relation: punct
Children: []

Word: A
Tag: DT
Head: cup
Dependency relation: det
Children: []

Word: red
Tag: JJ
Head: cup
Dependenc

In [8]:
def flatten_subtree(st):
    return ''.join([w.text_with_ws for w in list(st)]).strip()

# With this function in our toolbox, we can write a loop that prints out the subtree for each word in a sentence:

In [9]:
subjects_list = []
for word in doc:
    if word.dep_ == ('nsubj', 'nsubjpass'):
        subjects_list.append(flatten_subtree(word.subtree))
subjects_list

[]

In [10]:
nsubj_subjects = []
nsubj_count    = 0
for word in doc:
    if word.dep_ == ('nsubj'):
        nsubj_count += 1
        nsubj_subjects.append(flatten_subtree(word.subtree))
nsubj_subjects
nsubj_count

nsubjpass_subjects = []
nsubjpass_count = 0
for word in doc:
    if word.dep_ == ('nsubjpass'):
        nsubjpass_count += 1
        nsubjpass_subjects.append(flatten_subtree(word.subtree))
nsubjpass_subjects
nsubjpass_count

['I', 'I']

2

['The illness']

1

### Create the 2 x lists of positive & negative words from General Inquirer


In [11]:
%run sentiment5_general_inquirer_import.ipynb

#### print sample of first and last positive & negative words

In [12]:
print("Positive sample")
positive[:5]
positive[-5:]

print()
print("Negative sample")
negative[:5]
negative[-5:]

Positive sample


['advantage', 'stylish', 'appease', 'welfare', 'civility']

['exuberance', 'benefactor', 'conjure', 'steadfastness', 'standardize']


Negative sample


['pry', 'repulse', 'shoot', 'deprive', 'temper']

['illegality', 'uninformed', 'treachery', 'break', 'unreasonable']

### Define tokenizer and stemmer functions

In [13]:
%run -i sentiment_tokenize.py


#### Assert it is working

In [14]:
print("tokenize_only")
test = "Hi my friends"
tokenize_only(test)

print("open_and_tokenize")
##################  loop over this & increment counter +1 each time.
counter = 0
folder  = "sentiment_text_data"
open_and_tokenize(counter, folder)

# print("tokenize_and_stem")
# tokenize_and_stem(test)


tokenize_only


['hi', 'my', 'friends']

open_and_tokenize

File number: 1
File name: 1_1 PR.txt

Open file and display 'type'

Read file and display 'type'

Tokenise file and display 'type'

Just show the first & last 10 words of the main dialogue text file
These are first 10 words:
scenario
pos
aaa
d1
it
s
going
a
little
bit
These are last 10 words:
pr43
pr44
pr45
pr46
pr47
pr48
pr49
pr50
pr51
1_1.mp3


['scenario',
 'pos',
 'aaa',
 'd1',
 'it',
 's',
 'going',
 'a',
 'little',
 'bit',
 'fast',
 'alright',
 'can',
 'i',
 'have',
 'a',
 'listen',
 'to',
 'the',
 'back',
 'of',
 'the',
 'chest',
 'there',
 'at',
 'the',
 'at',
 'at',
 'the',
 'breathing',
 'p1',
 'okay.',
 'd1',
 'if',
 'you',
 'don',
 't',
 'mind',
 'we',
 'll',
 'just',
 'sit',
 'you',
 'forward',
 'there.',
 'n1',
 'we',
 'll',
 'just',
 'sit',
 'you',
 'forward',
 'there',
 'okay',
 'one',
 'two',
 'three',
 'off',
 'you',
 'go',
 'that',
 's',
 'it',
 'd1',
 'if',
 'you',
 'nice',
 'deep',
 'breaths',
 'in',
 'and',
 'out',
 'please',
 'pause',
 'n1',
 'okay',
 'd1',
 'okay',
 'n1',
 'those',
 'obs',
 'pr1',
 'there',
 'still',
 'so',
 'we',
 'have',
 'a',
 'heart',
 'rate',
 'one',
 'twenty-eight',
 'sats',
 'of',
 'going',
 'between',
 'ninety-three',
 'and',
 'ninety-five',
 'and',
 'a',
 'blood',
 'pressure',
 'of',
 'one',
 'eleven',
 'eighty-one',
 'd1',
 'okay',
 'n1',
 'yeah',
 'd1',
 'eh',
 'i',
 'might',


### Create empty lists to contain the results


##### Speakers & Turns
7. no. of speakers for each dialogue
8. no. turns per speaker
9. no. words per speaker
14. mean no. words per turn
15. total turns per dialogue
16. total words per dialogue
17. Perc. total turns uttered by each speaker
18. Perc. total words uttered by each speaker
26. mean sentence length per speaker (do Men use shorter sentences? some research suggests this?)

##### Positive & Negative Sentiment
1. set of all positive words 
2. set of all negative words
3. frequency of pos words (i.e. frequency distribution)
4. frequency of neg words (i.e. frequency distribution)
5. pos score for each dialogue
6. neg score for each dialogue
10. Perc. words that are positive
11. Perc. words that are negative
21. Positive word per speaker
22. Negative word per speaker

##### Active & Passive phrases
12. Perc. words that are active
13. Perc. words that are passive
19. ACtive word per speaker
20. Passive word per speaker

##### Parts of Speech
23. adverbs per speaker (e.g. Women elaborate more than men - women use adverbs more)
24. adjectives per speaker
25. freq of word "I" per speaker.

##### Medical terms
27. count medical terminology / concepts / jargon per speaker

##### Alignment / repetition / mirroring
28. mirroring of communication style (e.g. gender effects)



In [15]:
%run -i sentiment_create_scores.py

In [16]:
### show some of these empty lists that i created:

positive_score_list
pos_set_all_dialogue
Num_turns_P1

[]

[]

[]

#### Check number of files present in the directory

In [17]:

print("Check how many files in the directory")
### Check how many files in the directory
no_files = len(os.listdir(folder))
no_files

Check how many files in the directory


8

# Part 2: Loop (i.e. repeat for every text file in folder)


###  Specific words: occurrence count


In [18]:
tokens = open_and_tokenize(counter, folder)
doc = nlp(str(tokens))



### "list_words" --- user input from the beginning

dictionary_word_count = {} 

for x in  list_words:
        dictionary_word_count["word_{0}_count".format(x)] = 0


for word in doc:
    for x in  list_words:
        if str(word) == x:
            dictionary_word_count["word_{0}_count".format(x)] += 1

for x in  list_words:
    print("word_{0}_count".format(x))
    print(dictionary_word_count["word_{0}_count".format(x)])
    print()



File number: 1
File name: 1_1 PR.txt

Open file and display 'type'

Read file and display 'type'

Tokenise file and display 'type'

Just show the first & last 10 words of the main dialogue text file
These are first 10 words:
scenario
pos
aaa
d1
it
s
going
a
little
bit
These are last 10 words:
pr43
pr44
pr45
pr46
pr47
pr48
pr49
pr50
pr51
1_1.mp3
word_i_count
110

word_you_count
137

word_we_count
45

word_decide_count
0

word_decision_count
0

word_option_count
1

word_options_count
0



In [36]:
tokens = open_and_tokenize(counter, folder)

doc = nlp(str(tokens))



###################################################################  Active / Passive
nsubj_subjects = []
nsubj_count    = 0
for word in doc:
    if word.dep_ == ('nsubj'):
        nsubj_count += 1
        nsubj_subjects.append(flatten_subtree(word.subtree))
nsubj_subjects
nsubj_count

nsubjpass_subjects = []
nsubjpass_count = 0
for word in doc:
    if word.dep_ == ('nsubjpass'):
        nsubjpass_count += 1
        nsubjpass_subjects.append(flatten_subtree(word.subtree))
nsubjpass_subjects
nsubjpass_count

################################################################### Parts of speech:
pos = {}

mylist = ["verb", "personal_pronoun", "adverb", "adjective"]

for x in mylist:
    pos["{0}_list".format(x)] = ''
    pos["{0}_count".format(x)] = 0

pos

temp_list = []
for word in doc:
    if word.tag_ == ('VBD'):
        pos["verb_count"]  +=  1
        temp_list.append(flatten_subtree(word.subtree))
pos["verb_list"]  = temp_list


temp_list = []
for word in doc:
    if word.tag_ == ('PRP'):
        pos["personal_pronoun_count"]  +=  1
        temp_list.append(flatten_subtree(word.subtree))
        pos["personal_pronoun_list"]  = personal_pronoun_list

        
print("verb_count")
pos["verb_count"]
print("")

print("verb_list")
pos["verb_list"]
print("")

print("personal_pronoun_count")
pos["personal_pronoun_count"]
print("")

print("personal_pronoun_list")
pos["personal_pronoun_list"]


"""
### adverb
word.tag_ == ('RB'):

### adjective
word.tag_ == ('JJ'):
"""

    
################################################################### Phrases specified by Robin:


    


File number: 1
File name: 1_1 PR.txt

Open file and display 'type'

Read file and display 'type'

Tokenise file and display 'type'

Just show the first & last 10 words of the main dialogue text file
These are first 10 words:
scenario
pos
aaa
d1
it
s
going
a
little
bit
These are last 10 words:
pr43
pr44
pr45
pr46
pr47
pr48
pr49
pr50
pr51
1_1.mp3


["'the",
 'that',
 'you',
 'that',
 "when', 'did', 'you', 'have', 'the', 'operation', 'mrs', 'o', 'shaughnessy', 'p1', 'oh', 'it',",
 "'p1', 'three', 'days', 'ago', 'd1', 'and'",
 "'in', 'my', 'tummy', 'i', 'think', 'd1', 'alright', 'okay', 'well', 'i', 'just', 'want', 'to', 'get', 'a', 'tracing', 'of', 'the', 'heart', 'and', 'then', 'i', 'might', 'eh', 'examine', 'the', 'tummy', 'if', 'that', 's', 'okay', 'with', 'you.', 'p1', 'did', 'everything', 'go',",
 "'okay', 'd1', 'eh', '",
 "your', 'heart', '",
 "'i', 'm',",
 "'mrs', 'o', 'shaughnessy', 'p1', 'just', '",
 "'s', 'going', 'on', 'd1', 'i', 'd', 'like', 'to',",
 "'okay', 'd1', 'and', 'a', 'coag', 'as', 'well', 'n1', 'coag', 'as', 'well', 'i', 'see', 'one', 'of', 'your', 'colleagues', 'on', 'the', 'corridor', 'there', 'i', 'might', 'ask', 'him', 'to', 'pop', 'in', 'to', 'you', 'd1', 'yeah', 'would', 'you', 'mind', 'yeah', 'how', 'old',",
 "'i'",
 ", 'doctor', 'd2', 'hi', 'pr3', 'doctor', 'd2', 'how', 'are', 'you', 'inaudible', 'pr4

142

[]

0

...


personal_pronoun_count


466


personal_pronoun_list


['it',
 "i'",
 'you',
 "we', 'll'",
 'you',
 'we',
 "you'",
 'you',
 'it',
 'you',
 "'we',",
 "'i', 'might'",
 'you',
 "'i',",
 'you',
 "'i',",
 "'i'",
 "i', 'll'",
 'you',
 'it',
 'it',
 "'i'",
 "'i', 'just'",
 "'i'",
 'you',
 'we',
 "'i', 'm',",
 "'i', 'll'",
 'you',
 "'we'",
 "you'",
 'you',
 'i',
 "'i'",
 'i',
 'him',
 'you',
 "you'",
 'you',
 "'i', 'am', 'sixty-eight', 'd1', 'sixty-eight', 'p1', 'years', 'young', 'pause'",
 "'i'",
 'you',
 'she',
 "'she'",
 "she', 's'",
 'em',
 "'her",
 "'her',",
 'you',
 "'you'",
 "'we', 'have', 'a', 'full', 'blood', 'count', 'smac', 'twenty', 'eh', 'cop', 'cultures', 'and', 'coag'",
 'her',
 'you',
 'you',
 "'you', 'rather', 'me'",
 "'me'",
 'you',
 "'i',",
 "'i',",
 "'i'",
 "'you'",
 'you',
 "we'",
 'you',
 'you',
 'she',
 "she'",
 'em',
 "i', 'have', 'her', 'just'",
 "'her', 'just'",
 'you',
 'she',
 "she'",
 "i'",
 'she',
 'you',
 'she',
 'we',
 'you',
 'it',
 "'i', 'd', 'like…', 'p1', 'i', 'm', 'finding', 'it', 'hard', 'to', 'breathe.'",
 "'

"\n### adverb\nword.tag_ == ('RB'):\n\n### adjective\nword.tag_ == ('JJ'):\n"

{'adjective_count': 0,
 'adjective_list': '',
 'adverb_count': 0,
 'adverb_list': '',
 'personal_pronoun_count': 0,
 'personal_pronoun_list': '',
 'verb_count': 0,
 'verb_list': ''}

In [24]:
pos= {}
pos["verb_list"]        = ""
pos

{'verb_list': ''}

### Analyse content - use procedural code to avoid issues re local / global variables in functional programming

In [ ]:
##  %run -i sentiment_build_functions.py

counter = 0

# while counter < no_files:
while counter < no_files:
    tokens = open_and_tokenize(counter, folder)

    ############################################################  Number speakers & words & turns
    %run -i count_num_speakers.py
    """
    for content of 'tokens' - add 1 number to end of list "Num_speakers"
    """

    %run -i turns_per_speaker.py
    """
    For tokens - add 1 x number to variable for each speaker e.g. "Num_turns_P1"
    """

    ############################################################  Positive & Negative sentiment
    %run -i pos_neg_score_append.py
    """
    DONE FOR LISTS - MAYBE TRY AS A DICTIONARY?
    count number of pos & neg words in each transcript.
    for content of 'tokens' - add 1 number to positive_score_list & negative_score_list
    """
    
    %run -i pos_neg_words_set_freq.py
    """
    count freq of all pos / neg words across all files.
    create a set showing each pos / neg word once
    """
    
    ############################################################  Parts of speech
    %run -i parts_of_speech.py
    """
    
    """
    
    ############################################################  Active & Passive phrases
    %run -i spacy_dep_parse.py
    """
    Active / Passive phrases
    """
    
    ############################################################  Alignment / Repetition / Mirroring
    %run -i align_package_nickduran_pr.py
    """
    Run Nick Duran's 'align' package on the data.
    """
    
    ############################################################  Medical terminology
    %run -i count_med_terms.py
    """
    Count the number of medical concepts
    """
    
    
    counter += 1



print(positive_num)
print(negative_num)
print(positive_score_list)
print(negative_score_list)

print(len(pos_set_all_dialogue))
print(pos_set_all_dialogue)
print(len(neg_set_all_dialogue))
print(neg_set_all_dialogue)

x = nltk.FreqDist(pos_freq_all_dialogue)
x
x = nltk.FreqDist(neg_freq_all_dialogue)
x

Num_speakers

Num_turns_P1

### Assert vectors are all the same length

In [ ]:
assert len() == len()  & /
       len() == len()

#### Assert the functions work

# Part 2: Loop (i.e. repeat for every text file in folder)
### open the first text file: tokenise it.

In [ ]:
### First: define name of file to open
file = os.listdir("sentiment_text_data")[0]
type(file)
file

print()
print("Open file and display 'type'")
f = open(file)
type(f)

print()
print("Read file and display 'type'")
f2   = f.read()
type(f2)

print()
print("Tokenise file and display 'type'")
### Tokenize the text:
tokens = tokenize_only(f2)
type(tokens)

################################################## run the functions from the other file.
loop_functions()

In [ ]:
counter = 0

# while counter < no_files:
open_and_tokenize()
tokens[:10]
tokens[-10:]

pos_neg_lists_append_func()

pos_in_this, neg_in_this,  pos_freq_in_this, neg_freq_in_this  =   pos_neg_words_this_file_func()

print("1")
pos_set_all_dialogue
print("2")
pos_set_all_dialogue.extend(pos_in_this)
print("3")
pos_set_all_dialogue

In [ ]:
extend_set_all_pos_neg_words_func()

"""
count_num_speakers_func()
turns_per_speaker_func()   
counter = counter + 1
"""

# Part 3: after loop

#### Positive & Negative word set, Pos & Neg frequency distribution

In [ ]:
pos_set_all_dialogue
%run -i sentiment_pos_neg_results.py

In [ ]:
sentiment_more_results()

#### Turn taking: number of words and sentences per speaker

In [ ]:
%run sentiment_turn_taking_results.py

#### Active & Passive noun dependencies

In [ ]:
%run sentiment_active_passive.py